# QSAR/QSPR Models Fine-Tuning 1: Classification 

This notebook is an example of a classification task on BBBP dataset.

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*') # switch off RDKit warning messages

import pickle

from fastai import *
from fastai.text import *
from utils import *

torch.cuda.set_device(1) #change to 0 if you only has one GPU 

In [3]:
data_path = Path('../results')
name = 'BBBP'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

## Load Data

In [4]:
bbbp_data = pd.read_csv('../data/QSAR/bbbp.csv')
print('Dataset:', bbbp_data.shape)
bbbp_data.head(1)

Dataset: (2039, 2)


,smiles,p_np
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,1


We benchmarked our MolPMoFiT method to other published models from [Yang et al](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.9b00237) on
three well-studied datasets: **lipophilicity**, **HIV** and **BBBP**. All the models were evaluated on the
same ten 80:10:10 [splits](https://github.com/swansonk14/chemprop/blob/master/splits.tar.gz) from Yang et al to ensure a fair and reproducible benchmark.


In [9]:
# Change the split type and id to nagivate different splits.
dataset = 'bbbp'
split_type = 'random'
split_id = 11

split_file = f'{dataset}/{split_type}/split_indices{split_id}.pckl'

with open(f'../data/QSAR/splits/{split_file}', 'rb') as f:
    split = pickle.load(f)
    
print('Train Set:', len(split[0]))
print('Valid Set:', len(split[1]))
print('Test Set:', len(split[2]))

Train Set: 1631
Valid Set: 204
Test Set: 204


In [10]:
train = bbbp_data.iloc[split[0]]
valid = bbbp_data.iloc[split[1]]
test = bbbp_data.iloc[split[2]]
print('Positive Sample:',np.sum(train.p_np == 1), np.sum(valid.p_np == 1), np.sum(test.p_np == 1))

Positive Sample: 1259 151 150


## Data Augmentation

In [11]:
def bbbp_smiles_augmentation(df, N_rounds):
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        if df.iloc[i].p_np == 1:
            for j in range(N_rounds[0]):
                dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
                dist_aug['p_np'].append(df.iloc[i]['p_np'])

        if df.iloc[i].p_np == 0:
            for j in range(N_rounds[1]):
                dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
                dist_aug['p_np'].append(df.iloc[i]['p_np'])
        
    df_aug = pd.DataFrame.from_dict(dist_aug)
    df_aug = df_aug.append(df, ignore_index=True)
    return df_aug.drop_duplicates('smiles')

As shown above, the dataset is not balanced. For training data, we generated 10 and 30 randomized SMILES for molecules belong to the positive and negative classes, respectively. The numbers can be changed based on different datasets.

In [14]:
train_aug = bbbp_smiles_augmentation(train, [10,30])
valid_aug = bbbp_smiles_augmentation(valid, [5,5])

print('Train_aug Samples:', train_aug.shape[0])
print('Positive:Negative',np.sum(train_aug.p_np == 1)/ np.sum(train_aug.p_np == 0))

print('Valid_aug Samples:', valid_aug.shape[0])
print('Positive:Negative',np.sum(valid_aug.p_np == 1)/ np.sum(valid_aug.p_np == 0))

Train_aug Samples: 23873
Positive:Negative 1.1944112510341025
Valid_aug Samples: 1195
Positive:Negative 2.805732484076433


## Adpot the Encoder of MSPM According to the Target Dataset.

In order to fine-tuning the pre-trained MSPM on the QSAR datasets of interest, we need to prepare the data:

- Tokenize the SMILES of the QSAR dataset.
- Align the token IDs of the QSAR dataset to the token IDs pre-trained MSPM. 

Often, the vocab size of the QSAR dataset is different from that of the pre-trained strcuture prediction model, which means the QSAR model will have a different input size from that of pre-trained model. Here, we need to change the input size of the pre-trained model to the vocab size of the QSAR dataset.

In [26]:
bs = 128
tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])

In [27]:
qsar_vocab = TextLMDataBunch.from_df(path, train_aug, valid_aug, bs=bs, tokenizer=tok, 
                              chunksize=50000, text_cols=0,label_cols=1, max_vocab=60000, include_bos=False)

print(f'Vocab Size: {len(qsar_vocab.vocab.itos)}')

Vocab Size: 48


In [28]:
pretrained_model_path = Path('../results/MSPM/models')

pretrained_fnames = ['MSPM_wt', 'MSPM_vocab']
fnames = [pretrained_model_path/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]

In [31]:
lm_learner = language_model_learner(qsar_vocab, AWD_LSTM, drop_mult=1.0)
lm_learner = lm_learner.load_pretrained(*fnames)
lm_learner.freeze()
lm_learner.save_encoder(f'lm_encoder')

## Databunch for QSAR Modeling

You need to change the `text_cols` and `label_col` based on your dataset.

In [30]:
data_clas = TextClasDataBunch.from_df(path, train_aug, valid_aug, bs=bs, tokenizer=tok, 
                                          chunksize=50000, text_cols='smiles',label_cols='p_np', 
                                          vocab=qsar_vocab.vocab, max_vocab=60000, include_bos=False)

## Fine-tuning

In [33]:
cls_learner = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.2, callback_fns=AUROC)
cls_learner.load_encoder(f'lm_encoder')
cls_learner.freeze()

In [35]:
cls_learner.fit_one_cycle(4, 3e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,AUROC,time
0,0.407379,0.460653,0.797490,0.884794,00:18
1,0.318413,0.455647,0.834310,0.882303,00:21
2,0.270218,0.396474,0.854393,0.913720,00:20
3,0.204793,0.370928,0.876987,0.917120,00:21


In [36]:
cls_learner.freeze_to(-2)
cls_learner.fit_one_cycle(4, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,AUROC,time
0,0.209718,0.395000,0.871130,0.912704,00:25
1,0.164111,0.451916,0.882845,0.915477,00:25
2,0.100156,0.424936,0.890377,0.926186,00:26
3,0.077003,0.429915,0.888703,0.922358,00:25


In [37]:
cls_learner.freeze_to(-3)
cls_learner.fit_one_cycle(4, slice(5e-4/(2.6**4),5e-4), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,AUROC,time
0,0.081082,0.440379,0.889540,0.922670,00:44
1,0.070148,0.460488,0.888703,0.917572,00:43
2,0.095224,0.472792,0.887866,0.923655,00:39
3,0.081094,0.458418,0.888703,0.925186,00:39


In [38]:
cls_learner.unfreeze()
cls_learner.fit_one_cycle(6, slice(5e-5/(2.6**4),5e-5), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,AUROC,time
0,0.093497,0.467894,0.888703,0.920641,00:50
1,0.094730,0.486960,0.887866,0.919988,00:50
2,0.094285,0.450275,0.892887,0.922824,00:51
3,0.068093,0.469609,0.883682,0.921969,00:56
4,0.063156,0.459041,0.887866,0.923721,00:53
5,0.057931,0.449040,0.886192,0.925472,01:00


In [39]:
cls_learner.save(f'{split_type}_{split_id}_clas')

## Test on the Test Set

1. Test only on Canoicial SMILES

In [45]:
test_data_clas = TextClasDataBunch.from_df(path, train, test, bs=bs, tokenizer=tok, 
                              chunksize=50000, text_cols='smiles',label_cols='p_np', vocab=qsar_vocab.vocab, max_vocab=60000,
                                              include_bos=False)

learner = text_classifier_learner(test_data_clas, AWD_LSTM, pretrained=False, drop_mult=0.2)
learner.load(f'{split_type}_{split_id}_clas', purge=False);

In [77]:
test_get_scores(learner)

Testing 204 molecues
Accuracy: 0.922
False Positives: 0.049
False Negatives: 0.029
Recall: 0.960
Precision: 0.935
Sensitivity: 0.960
Specificity: 0.815
MCC: 0.795
ROCAUC: 0.943


2. Test on averaging prediction of canoicial and randomized SMILES.

In [47]:
def test_smiles_augmentation(df):
    dist_aug = {col_name: [] for col_name in df}
    
    for i in range(df.shape[0]):
        dist_aug['smiles'].append(randomize_smiles(df.iloc[i]['smiles']))
        dist_aug['p_np'].append(df.iloc[i]['p_np'])
                     
    return pd.DataFrame.from_dict(dist_aug)

In [48]:
lb = torch.tensor(test['p_np'].values)

In [68]:
preds = []

# Randomized SMILES Predictions
for i in range(4):
    np.random.seed(12*i)    
    test_aug = test_smiles_augmentation(test)
    
    # model
    test_data_clas = TextClasDataBunch.from_df(path, train, test_aug, bs=bs, tokenizer=tok, 
                              chunksize=50000, text_cols='smiles',label_cols='p_np', vocab=qsar_vocab.vocab, max_vocab=60000,
                                              include_bos=False)
    learner = text_classifier_learner(test_data_clas, AWD_LSTM, pretrained=False, drop_mult=0.2)
    learner.load(f'{split_type}_{split_id}_clas', purge=False);
    
    
    #get predictions
    pred,lbl = learner.get_preds(ordered=True)
    
    preds.append(pred)

# Canonical SMILES Predictions

test_data_clas = TextClasDataBunch.from_df(path, train, test, bs=bs, tokenizer=tok, 
                              chunksize=50000, text_cols='smiles',label_cols='p_np', vocab=qsar_vocab.vocab, max_vocab=60000,
                                              include_bos=False)

learner = text_classifier_learner(test_data_clas, AWD_LSTM, pretrained=False, drop_mult=0.2)
learner.load(f'{split_type}_{split_id}_clas', purge=False);


pred,lbl = learner.get_preds(ordered=True)


preds.append(pred)



In [79]:
avg_preds = sum(preds)/len(preds)
print(f'Performance of Averaging Predictions of Canoicial and Randomized SMILES: {roc_auc_score(lbl, avg_preds[:,1]):.3f}')

Performance of Averaging Predictions of Canoicial and Randomized SMILES: 0.943
